# Étape 2 – Baseline frame-wise (pilotage Colab)

Ce notebook sert uniquement à **piloter l'entraînement de la baseline frame-wise** sur Google Colab / Colab Pro+.

La logique d'entraînement (DataLoaders, modèle, pertes, optimiseur, etc.) est implémentée dans `src/`.
Le notebook se contente de :

1. Monter Google Drive (`MyDrive/QV-Pipe/`).
2. Cloner ou mettre à jour le dépôt du projet dans `/content`.
3. Installer les dépendances Python.
4. Définir les chemins vers les données (frames, splits, labels).
5. Lancer le script d'entraînement `src/train/framewise_baseline.py`.


In [1]:
# Vérifier le GPU Colab disponible
!nvidia-smi

Wed Nov 26 21:35:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             49W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed